In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


steps:

alice and bob want to establish a shared key made of N bits.

## requirements
start with a sequence of entangled pairs of qbits
- could've been sent by a 3rd party
- each pair is in state 1/sq2(|01> - |10>)

## testing entanglement (see lecture 8 & lab 4B)
to test the entanglement, (for each pair)
1. alice chooses randomly Z or X.
2. bob chooses randomly W = 1/sq2(X+Z) or V = 1/sq2(X-Z)
3. Alice and Bob measure $S = |\langle X \otimes W \rangle - \langle X \otimes V \rangle + \langle Z \otimes W \rangle + \langle Z \otimes V \rangle|$
- $\langle X \otimes W \rangle$ is the avg value of the result of measuring.
- measuring means:
    1. measuring Alice's qbit in the basis corresponding to X **and**
    2. measuring Bob's in the basis corresponding to W

what is the basis coresponding to X or W?

4. the result is converted {0, 1} $\rightarrow$ {+1, -1}
5. these values are multiplied together

after doing this **for each pair**,
the average values from that are added and then divided by the length of the sequence.



bob and alice never measure in the same basis as each other,
so they're never guaranteed to get the same result from both of their measurements.


IF THE BASIS IS THE SAME.. THE RESULT IS GUARANTEED THE SAME

In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.

def show_latex(circuit, bits):
    state = Statevector.from_int(0, 2**bits)
    state = state.evolve(circuit)
    display(state.draw("latex"))

def randomProb(render=False, ZeroStat=1/2, phi=math.pi/2):
    # note: this is is radians.
    theta = 2 * math.acos(math.sqrt(ZeroStat) )    
    
    circuit = QuantumCircuit(1)
    circuit.r(theta, phi, 0)
    
    # small interrupt to render latex for our state
    if (render):
        state = Statevector.from_int(0, 2**1)
        state = state.evolve(circuit)
        display(state.draw("latex"))
    
    circuit.measure_all()
    
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    results_sim = job_sim.result()
    counts = results_sim.get_counts()
    return int(list(counts.keys())[0])

def random50(render=False):
    return randomProb(render, 1/2, math.pi/2)

def random33(render=False):
    """
    -ie^-io sin(theta/2) = 2/sq(3)
    theta = 1.91
    i*-ie^-io  = i *  2/sq(3) / sin(1.91/2)
    -io = ln(i *  2/sq(3) / sin(1.91/2))
    o = ln(i *  2/sq(3) / sin(1.91/2)) / -i
    """
    # math.log(i * 2/math.sqrt(3) / math.sin(theta/2), math.e)
    # TODO ^ COME BACK TO THIS
    
    return randomProb(render, 1/3, phi=math.pi/2)


random50(True)
random33(True)
# result = [random33() for i in range(1)]
# print(result)
# print(sum(result)/len(result))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

1

In [17]:
############################################# OWN CONSTANTS #############################################

# divRoot2 = 1/math.sqrt(2)

# W = [ [-divRoot2, -divRoot2],
#       [-divRoot2,  divRoot2] ]

# V = [ [-divRoot2, divRoot2],
#       [ divRoot2, divRoot2] ]

############################################# LAB CONSTANTS #############################################

root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 


W = [ [ -1 / denom1 , (1 + root2) / denom1 ],
      [  1 / denom2 , (root2 - 1) / denom2 ] ]

V = [ [  1 / denom1 , (1 + root2) / denom1 ],
      [ -1 / denom2 , (root2 - 1) / denom2 ] ]
V=B1_transform_matrix
############################################# CODE #######################################################


targetKeyLen = 100

finalKey = []
S = {(1,1):[0, 0], (1,3):[0, 0], (3,1):[0, 0], (3,3):[0, 0]}

for qbitI in range(int(targetKeyLen * 9/2)):
    # first, create the entangled pair
    # circuit_to_gate

    # build the bell state
    circuit = QuantumCircuit(2)
    circuit.x(0)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.x(0)

    # show_latex(circuit, 2)
    # display(circuit.draw("mpl"))

    # Alice chooses an operator Ai randomly from her set of three
    if random33() == 0:
        i = 1
        # circuit.x(1)
    elif random50() == 0:
        i = 2        
        circuit.unitary(W, [1])
    else:
        i = 3
        # circuit.z(1)
        circuit.h(1)

    # Bob chooses an operator Bj randomly from his set of three
    if random33() == 0:
        j = 1
        circuit.unitary(W, [0])
    elif random50() == 0:
        j = 2
        # circuit.z(0)
        circuit.h(1)
    else:
        j = 3
        circuit.unitary(V, [0])
    
    # bob agreed to invert his bits
    circuit.x(0)

    # both alice and bob measure their qbits.
    circuit.measure_all()
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    results_sim = job_sim.result()
    counts = results_sim.get_counts()
    measurements = list(counts.keys())

    # operator choices are shared now.

    if (i == 3 and j == 2) or (i == 2 and j == 1):
        finalKey.append(measurements)
    elif (i == 1 and j == 2) or (i == 2 and j == 2) or (i == 2 and j == 3):
        pass
    else:
        key = (i, j)
        measurements = [int(val) * -2 + 1 for val in measurements[0]]
        newVal = S[key][0] + measurements[0] * measurements[1]
        newFreq = S[key][1] + 1
        S[i, j] = [newVal, newFreq]

    
    # display(circuit.draw("mpl"))

for key in S:
    if S[key] == [0, 0]:
        S[key] = [0, 1]

print(f"key ratio: {int(targetKeyLen * 9/2)/len(finalKey)}, ideally {9/2=}")
print(f"final key (len {len(finalKey)}):")
print(finalKey)
print()
print("S:")
print(S)

print()
print("S components:")
print(f"{S[(1, 1)][0]} / {S[(1, 1)][1]} = {S[(1, 1)][0] / S[(1, 1)][1]}")
print(f"{S[(1, 3)][0]} / {S[(1, 3)][1]} = {S[(1, 3)][0] / S[(1, 3)][1]}")
print(f"{S[(3, 1)][0]} / {S[(3, 1)][1]} = {S[(3, 1)][0] / S[(3, 1)][1]}")
print(f"{S[(3, 3)][0]} / {S[(3, 3)][1]} = {S[(3, 3)][0] / S[(3, 3)][1]}")

print()
print("S sum:")
Ssum = abs((S[(1, 1)][0] / S[(1, 1)][1]) +
            (S[(1, 3)][0] / S[(1, 3)][1]) +
            (S[(3, 1)][0] / S[(3, 1)][1]) -
            (S[(3, 3)][0] / S[(3, 3)][1]))

print()
print(f"{Ssum=}, {2 * 2**0.5=}")

key ratio: 5.172413793103448, ideally 9/2=4.5
final key (len 87):
[['11'], ['11'], ['00'], ['00'], ['00'], ['00'], ['00'], ['11'], ['00'], ['11'], ['00'], ['11'], ['11'], ['00'], ['00'], ['11'], ['11'], ['11'], ['11'], ['00'], ['00'], ['00'], ['11'], ['11'], ['11'], ['11'], ['11'], ['11'], ['00'], ['00'], ['11'], ['11'], ['11'], ['00'], ['11'], ['11'], ['11'], ['00'], ['11'], ['11'], ['00'], ['00'], ['00'], ['00'], ['11'], ['00'], ['00'], ['11'], ['11'], ['00'], ['00'], ['11'], ['00'], ['00'], ['00'], ['00'], ['11'], ['11'], ['11'], ['00'], ['11'], ['00'], ['00'], ['11'], ['11'], ['00'], ['11'], ['00'], ['11'], ['00'], ['11'], ['00'], ['00'], ['11'], ['11'], ['11'], ['00'], ['00'], ['00'], ['11'], ['11'], ['00'], ['00'], ['11'], ['11'], ['11'], ['00']]

S:
{(1, 1): [1, 49], (1, 3): [-1, 43], (3, 1): [51, 51], (3, 3): [-53, 53]}

S components:
1 / 49 = 0.02040816326530612
-1 / 43 = -0.023255813953488372
51 / 51 = 1.0
-53 / 53 = -1.0

S sum:

Ssum=1.9971523493118177, 2 * 2**0.5=2.8284271

In [12]:

# Useful constants
root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 


B0_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]
W=B0_transform_matrix

B1_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]
V=B1_transform_matrix

targetKeyLen = 100

S = {(1,1):[0, 0], (1,2):[0, 0], (2,1):[0, 0], (2,2):[0, 0]}
Scount = 0

for qbitI in range(targetKeyLen):
    # first, create the entangled pair
    # circuit_to_gate

    # build the bell state
    circuit = QuantumCircuit(2) 
    circuit.h(0)
    circuit.cx(0,1)
    
    # Alice chooses an operator Ai randomly from her set of three
    if random50() == 0:
        i = 1
    else:
        i = 2        
        circuit.h(1)

    # Bob chooses an operator Bj randomly from his set of three
    if random50() == 0:
        j = 1
        circuit.unitary(W, [0])
    else:
        j = 2
        circuit.unitary(V, [0])
    
    # both alice and bob measure their qbits.
    circuit.measure_all()
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    results_sim = job_sim.result()
    counts = results_sim.get_counts()
    measurements = list(counts.keys())

    # operator choices are shared now.

    key = (i, j)
    measurements = [int(val) * -2 + 1 for val in measurements[0]]
    newVal = S[key][0] + measurements[0] * measurements[1]
    newFreq = S[key][1] + 1
    S[i, j] = [newVal, newFreq]
    Scount += 1

for key in S:
    if S[key] == [0, 0]:
        S[key] = [0, 1]

print()
print("S:")
print(S)

print()
print("S components:")
print(f"{S[(1, 1)][0]} / {S[(1, 1)][1]} = {S[(1, 1)][0] / S[(1, 1)][1]}")
print(f"{S[(1, 2)][0]} / {S[(1, 2)][1]} = {S[(1, 2)][0] / S[(1, 2)][1]}")
print(f"{S[(2, 1)][0]} / {S[(2, 1)][1]} = {S[(2, 1)][0] / S[(2, 1)][1]}")
print(f"{S[(2, 2)][0]} / {S[(2, 2)][1]} = {S[(2, 2)][0] / S[(2, 2)][1]}")

print()
print("S sum:")
Ssum = abs((S[(1, 1)][0] / S[(1, 1)][1]) +
            (S[(1, 2)][0] / S[(1, 2)][1]) +
            (S[(2, 1)][0] / S[(2, 1)][1]) -
            (S[(2, 2)][0] / S[(2, 2)][1]))

print()
print(f"{Ssum=}, {2 * 2**0.5=}")


S:
{(1, 1): [-19, 29], (1, 2): [-11, 21], (2, 1): [-16, 22], (2, 2): [16, 28]}

S components:
-19 / 29 = -0.6551724137931034
-11 / 21 = -0.5238095238095238
-16 / 22 = -0.7272727272727273
16 / 28 = 0.5714285714285714

S sum:

Ssum=2.477683236303926, 2 * 2**0.5=2.8284271247461903


In [11]:
# Useful constants
root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 


B0_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]
W=B0_transform_matrix

B1_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]
V=B1_transform_matrix



divRoot2 = 1/math.sqrt(2)

W = [ [-divRoot2, -divRoot2],
      [-divRoot2,  divRoot2] ]

V = [ [-divRoot2, divRoot2],
      [ divRoot2, divRoot2] ]


targetKeyLen = 100

S = {(1,1):[0, 0], (1,2):[0, 0], (2,1):[0, 0], (2,2):[0, 0]}
Scount = 0

for qbitI in range(targetKeyLen):
    # first, create the entangled pair
    # circuit_to_gate

    # build the bell state
    circuit = QuantumCircuit(2) 
    circuit.h(0)
    circuit.cx(0,1)
    
    # Alice chooses an operator Ai randomly from her set of three
    if random50() == 0:
        i = 1 
        circuit.x(1)
    else:
        i = 2        
        # circuit.h(1) 
        circuit.z(1)

    # Bob chooses an operator Bj randomly from his set of three
    if random50() == 0:
        j = 1
        circuit.unitary(W, [0])
    else:
        j = 2
        circuit.unitary(V, [0])
    
    # both alice and bob measure their qbits.
    circuit.measure_all()
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    results_sim = job_sim.result()
    counts = results_sim.get_counts()
    measurements = list(counts.keys())

    # operator choices are shared now.

    key = (i, j)
    measurements = [int(val) * -2 + 1 for val in measurements[0]]
    newVal = S[key][0] + measurements[0] * measurements[1]
    newFreq = S[key][1] + 1
    S[i, j] = [newVal, newFreq]
    Scount += 1

for key in S:
    if S[key] == [0, 0]:
        S[key] = [0, 1]

print()
print("S:")
print(S)

print()
print("S components:")
print(f"{S[(1, 1)][0]} / {S[(1, 1)][1]} = {S[(1, 1)][0] / S[(1, 1)][1]}")
print(f"{S[(1, 2)][0]} / {S[(1, 2)][1]} = {S[(1, 2)][0] / S[(1, 2)][1]}")
print(f"{S[(2, 1)][0]} / {S[(2, 1)][1]} = {S[(2, 1)][0] / S[(2, 1)][1]}")
print(f"{S[(2, 2)][0]} / {S[(2, 2)][1]} = {S[(2, 2)][0] / S[(2, 2)][1]}")

print()
print("S sum:")
Ssum = abs((S[(1, 1)][0] / S[(1, 1)][1]) +
            (S[(1, 2)][0] / S[(1, 2)][1]) +
            (S[(2, 1)][0] / S[(2, 1)][1]) -
            (S[(2, 2)][0] / S[(2, 2)][1]))

print()
print(f"{Ssum=}, {2 * 2**0.5=}")


S:
{(1, 1): [-3, 21], (1, 2): [6, 30], (2, 1): [-4, 28], (2, 2): [-1, 21]}

S components:
-3 / 21 = -0.14285714285714285
6 / 30 = 0.2
-4 / 28 = -0.14285714285714285
-1 / 21 = -0.047619047619047616

S sum:

Ssum=0.03809523809523807, 2 * 2**0.5=2.8284271247461903
